In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob

In [ ]:
def calculate_xy(file, E_red, pH):
    """
    This function reads the txt file containing the LSV or CV data,
    extracts the relevant columns, and calculates the x and y values for plotting.
    The function takes the file path, E_red, and pH as inputs.
    It reads the file, skips the first 19 rows, and assigns column names.
    The function then extracts the 'potential' and 'current' columns. 
    The x in the fuction is E vs RHE (V) and y is the current density.
    """

    # Read the data file and skip the first 19 rows
    df = pd.read_csv(file, sep = "\\s+", skiprows = 19, header = None)

    # Assign column names
    df.columns = ["No", "time", "potential", "current"]

    # Calculate x and y values
    x = df["potential"] + E_red + (0.059 * pH)
    y = df["current"]/0.1979 

    # return x, y values
    return x, y

In [ ]:
print("""
This script calculates the x and y values for plotting LSV or CV data.
It takes the file path, E_red, and pH as inputs.
The script reads the file, skips the first 19 rows, and assigns column names.
      """)

In [ ]:
main_path = str(input("Enter the main path: ")) # instead of input, you can directly define the path here "D\nam\data\20250411"
folders = os.listdir(main_path) # Lists all the folders in the main path

In [ ]:
E_reds, pHs = [], [] # Define empty lists to store E_red and pH values

# Loop through each folder and get the E_red and pH values
for f in folders:
    # Go inside each folder and Ask the user for E_red and pH values for each folder
    E_red, pH = float(input(f"Enter the E_red value for {f} (Enter 0 if it is not needed for your calculation): ")), float(input(f"Enter the pH value for {f} (Enter 0 if it is not needed for your calculation): "))

    # Once you enter the values, append them to the lists
    E_reds.append(E_red)
    pHs.append(pH)

### If you dont want to view/save plots (Your main usecase for using these codes)

In [ ]:
# Define an empty dictionary to store the master data for LSV and CV
lsv_master_dict = {}
cv_master_dict = {}

# loop through each folder and read the LSV and CV files, while taking the E_red and pH values into account for the respective folder
for f, E_red, pH in zip(folders, E_reds, pHs):
    # Get the LSV and CV files in the folder
    # The glob module is used to find all the pathnames matching a specified pattern
    lsv_files = glob.glob(os.path.join(main_path, f, "*lsv*txt"))
    cv_files = glob.glob(os.path.join(main_path, f, "*cv*txt"))

    combined_lsv_dict = {}
    # Let us loop through the LSV files to read them one by one
    for lsv_file in lsv_files:

        # read the lsv file, and use our function to calculate x and y values (E vs RHE and current density)
        # The function calculate_xy is defined above
        x, y = calculate_xy(lsv_file, E_red, pH)

        # Save individual output as a data frame and convert it into a CSV file
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(lsv_file.replace('.txt', '_output.csv'), index=False)

        # Add to master dict
        combined_lsv_dict[f"{lsv_file.split('\\')[-1].split('.')[0]}_x"] = x
        combined_lsv_dict[f"{lsv_file.split('\\')[-1].split('.')[0]}_y"] = y

        # Read the file name, remove the .txt in the file name and replace it with filename_x and filename_y for the columns of the master data
        key_base = os.path.basename(lsv_file).replace('.txt', '')
        lsv_master_dict[f"{key_base}_x"] = x
        lsv_master_dict[f"{key_base}_y"] = y
    
    # Combine the LSV data into a single DataFrame
    combined_lsv_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in combined_lsv_dict.items()]))
    combined_lsv_df.to_csv(os.path.join(main_path, f, f"{f}_combined_LSV.csv"), index=False)

    

    # Same procedure for the CV files
    combined_cv_dict = {}
    for cv_file in cv_files:
        x, y = calculate_xy(cv_file, E_red, pH)

        # Save individual output
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(cv_file.replace('.txt', '_output.csv'), index=False)

        # Add to master dict
        key_base = os.path.basename(cv_file).replace('.txt', '')
        cv_master_dict[f"{key_base}_x"] = x
        cv_master_dict[f"{key_base}_y"] = y

        # combined cv dict
        combined_cv_dict[f"{cv_file.split('\\')[-1].split('.')[0]}_x"] = x
        combined_cv_dict[f"{cv_file.split('\\')[-1].split('.')[0]}_y"] = y

    # Combine the CV data into a single DataFrame
    combined_cv_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in combined_cv_dict.items()]))
    combined_cv_df.to_csv(os.path.join(main_path, f"{f}_combined_CV.csv"), index=False)


# Convert to DataFrames and save

# Convert LSV master dictionary to DataFrame and save
lsv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in lsv_master_dict.items()]))
lsv_master_df.to_csv(os.path.join(main_path, "master_LSV.csv"), index=False)

# Convert CV master dictionary to DataFrame and save
cv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in cv_master_dict.items()]))
cv_master_df.to_csv(os.path.join(main_path, "master_CV.csv"), index=False)

### If you want to view/save the plots

In [ ]:
lsv_master_dict = {}
cv_master_dict = {}

for f, E_red, pH in zip(folders, E_reds, pHs):
    lsv_files = glob.glob(os.path.join(main_path, f, "*lsv*.txt"))
    cv_files = glob.glob(os.path.join(main_path, f, "*cv*.txt"))


    combined_lsv_dict = {}
    combined_cv_dict = {}
    
    for lsv_file in lsv_files:
        x,y = calculate_xy(lsv_file, E_red, pH)
        plt.plot(x, y, label=lsv_file.split('\\')[-1].split('.')[0])
        plt.xlabel('E vs RHE (V)')
        plt.ylabel('Current Density')
        plt.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize=8)
        
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(lsv_file.replace('.txt', '_output.csv'), index=False)

        key_base = os.path.basename(lsv_file).replace('.txt', '')
        lsv_master_dict[f"{key_base}_x"] = x
        lsv_master_dict[f"{key_base}_y"] = y

        combined_lsv_dict[f"{lsv_file.split('\\')[-1].split('.')[0]}_x"] = x
        combined_lsv_dict[f"{lsv_file.split('\\')[-1].split('.')[0]}_y"] = y

    plt.grid()
    plt.title(f"Plot for {f}, LSV")
    plt.savefig(os.path.join(main_path, f, "lsv_plot.png"), dpi=300)
    plt.savefig(os.path.join(main_path, f, "lsv_plot.pdf"), dpi=300)
    plt.show()

    combined_lsv_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in combined_lsv_dict.items()]))
    combined_lsv_df.to_csv(os.path.join(main_path, f"{f}_combined_LSV.csv"), index=False)

    
    for cv_file in cv_files:
        x,y = calculate_xy(cv_file, E_red, pH)

        plt.plot(x, y, label=cv_file.split('\\')[-1].split('.')[0])
        plt.xlabel('E vs RHE (V)')
        plt.ylabel('Current Density')
        plt.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize=8)
        
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(cv_file.replace('.txt', '_output.csv'), index=False)

        key_base = os.path.basename(cv_file).replace('.txt', '')
        cv_master_dict[f"{key_base}_x"] = x
        cv_master_dict[f"{key_base}_y"] = y

        combined_cv_dict[f"{cv_file.split('\\')[-1].split('.')[0]}_x"] = x
        combined_cv_dict[f"{cv_file.split('\\')[-1].split('.')[0]}_y"] = y


    plt.grid()
    plt.title(f"Plot for {f}, CV")
    plt.savefig(os.path.join(main_path, f, "cv_plot.png"), dpi=300)
    plt.savefig(os.path.join(main_path, f, "cv_plot.pdf"), dpi=300)
    plt.show()

    # Combine the CV data into a single DataFrame
    combined_cv_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in combined_cv_dict.items()]))
    combined_cv_df.to_csv(os.path.join(main_path, f"{f}_combined_CV.csv"), index=False)


# Convert to DataFrames and save
lsv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in lsv_master_dict.items()]))
cv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in cv_master_dict.items()]))

lsv_master_df.to_csv(os.path.join(main_path, "master_LSV.csv"), index=False)
cv_master_df.to_csv(os.path.join(main_path, "master_CV.csv"), index=False)